In [1]:
%cd ~/RATER-D

/home/daved/RATER-D


In [2]:
import pandas as pd
import numpy as np
import jsonlines

In [3]:
train = pd.read_excel("data/raw/Highest domain-fixed.xlsx", sheet_name = 2)
pred_df = pd.read_excel("data/raw/Sample data sheet for Kai v9.xlsx")

human_sheet = pred_df.merge(train, on = 'Match ID', how = 'right')
human_sheet['Full construct'] = human_sheet['Construct name'] + ': ' + human_sheet['Construct definition']
human_sheet['target'] = human_sheet['H&T ratings']

train = human_sheet

In [4]:
thePerson = '''You are an academic expert. Please follow all the instructions very carefully. The questions are unique to survey measurement development and require detailed attention.

Research projects often use survey items to measure concepts. Examples in the management field include work motivation, job satisfaction, and employee stress. When writing survey items, researchers must take great care to ensure that the items do a good job of measuring the concepts of interest (e.g., that an item intended to measure work motivation really seems to capture that concept well). Your purpose is to assess survey items used in the various literatures (e.g., management).'''

thePrompt = '''### Your job is to assess the degree to which each survey item matches the concept statement provided.

You will be given a concept statement below, followed by a survey item. For each item, you will rate the degree to which it matches the provided concept statement.

Not all of the survey items will match the provided concept statement. Therefore, please pay close attention to each individual survey item as you decide whether it matches the provided concept statement.

### You will judge how well a survey item matches a particular statement using this response scale:
1. Item does an EXTREMELY BAD job of measuring the concept provided above
2. Item does a VERY BAD job of measuring the concept provided above
3. Item does a SOMEWHAT BAD job of measuring the concept provided above
4. Item does an ADEQUATE job of measuring the concept provided above
5. Item does a SOMEWHAT GOOD job of measuring the concept provided above
6. Item does a VERY GOOD job of measuring the concept provided above
7. Item does an EXTREMELY GOOD job of measuring the concept provided above

### The concept statement:
{statement}

### The survey item:
{item}

### Answer the question by immediately stating one response from the scale above verbatim. YOUR RESPONSE MUST MATCH THE SCALE EXACTLY WITHOUT ALTERATIONS, INCLUDING THE SCALE RESPONSE NUMBER.'''

print(thePrompt)

theJustification = 'LASTLY, PROVIDE A JUSTIFICATION FOR YOUR CHOICE.'
thePromptWithJustification = thePrompt + ' ' + theJustification

### Your job is to assess the degree to which each survey item matches the concept statement provided.

You will be given a concept statement below, followed by a survey item. For each item, you will rate the degree to which it matches the provided concept statement.

Not all of the survey items will match the provided concept statement. Therefore, please pay close attention to each individual survey item as you decide whether it matches the provided concept statement.

### You will judge how well a survey item matches a particular statement using this response scale:
1. Item does an EXTREMELY BAD job of measuring the concept provided above
2. Item does a VERY BAD job of measuring the concept provided above
3. Item does a SOMEWHAT BAD job of measuring the concept provided above
4. Item does an ADEQUATE job of measuring the concept provided above
5. Item does a SOMEWHAT GOOD job of measuring the concept provided above
6. Item does a VERY GOOD job of measuring the concept provided above


In [5]:
theScale = dict({
    1: '1. Item does an EXTREMELY BAD job of measuring the concept provided above',
    2: '2. Item does a VERY BAD job of measuring the concept provided above',
    3: '3. Item does a SOMEWHAT BAD job of measuring the concept provided above',
    4: '4. Item does an ADEQUATE job of measuring the concept provided above',
    5: '5. Item does a SOMEWHAT GOOD job of measuring the concept provided above',
    6: '6. Item does a VERY GOOD job of measuring the concept provided above',
    7: '7. Item does an EXTREMELY GOOD job of measuring the concept provided above'
})

In [6]:
prompts = []

for index, row in train.iterrows():
    prompt = thePrompt.format(
        statement = row['Construct definition'], 
        item = row['Item text']
        )
    prompts += [prompt]

In [7]:
print(prompts[0])

### Your job is to assess the degree to which each survey item matches the concept statement provided.

You will be given a concept statement below, followed by a survey item. For each item, you will rate the degree to which it matches the provided concept statement.

Not all of the survey items will match the provided concept statement. Therefore, please pay close attention to each individual survey item as you decide whether it matches the provided concept statement.

### You will judge how well a survey item matches a particular statement using this response scale:
1. Item does an EXTREMELY BAD job of measuring the concept provided above
2. Item does a VERY BAD job of measuring the concept provided above
3. Item does a SOMEWHAT BAD job of measuring the concept provided above
4. Item does an ADEQUATE job of measuring the concept provided above
5. Item does a SOMEWHAT GOOD job of measuring the concept provided above
6. Item does a VERY GOOD job of measuring the concept provided above


In [8]:
items = []

def binning_function (val, adjust_cutoffs = False):
    if adjust_cutoffs:
        cutoffs = [6.25, 5.5, 4.5, 3.5, 2.5, 1.75, 1]
        vals = [7, 6, 5, 4, 3, 2, 1]
        for i, each_cutoff in enumerate(cutoffs):
            if val >= each_cutoff:
                return vals[i]
    else:
        return np.rint(val)

for i, _ in enumerate(prompts):
    items.append([{"role": "system", "content": thePerson},
                  {"role": "user", "content": prompts[i]},
                  {"role": "assistant", "content": theScale[binning_function(train['target'][i])]}])

In [9]:
items[0]

[{'role': 'system',
  'content': 'You are an academic expert. Please follow all the instructions very carefully. The questions are unique to survey measurement development and require detailed attention.\n\nResearch projects often use survey items to measure concepts. Examples in the management field include work motivation, job satisfaction, and employee stress. When writing survey items, researchers must take great care to ensure that the items do a good job of measuring the concepts of interest (e.g., that an item intended to measure work motivation really seems to capture that concept well). Your purpose is to assess survey items used in the various literatures (e.g., management).'},
 {'role': 'user',
  'content': '### Your job is to assess the degree to which each survey item matches the concept statement provided.\n\nYou will be given a concept statement below, followed by a survey item. For each item, you will rate the degree to which it matches the provided concept statement.\n

In [10]:
with jsonlines.open('data/ft/datasets/train.txt', 'w') as writer:
    writer.write_all(items)

In [11]:
with jsonlines.open('data/ft/openai/train.jsonl', 'w') as writer:
    writer.write_all([{'messages': item} for item in items])